# Reconocimiento de Flores

## Práctica Final - Visión por computador
---
##### Autores: Marta Gómez Macías, Braulio Vargas López
---

### Definición del problema

El problema elegido consiste en extraer una serie de características del conjunto de imágenes para ser capaces de reconocer la especie de flor que es de las 17 especies posibles, mediante una imagen. Este problema consta de varias partes:

1. __Extracción de las características principales de la imagen__.
2. __Entrenar un algoritmo de aprendizaje a partir de las características extraídas__.

Para esto, vamos a hacer uso de distintos descriptores, como son el descriptor *HOG*, *Shape-Context* o el descriptor de SIFT.

#### Descriptor de HOG

El descriptor HOG, conocido como ___Histogram of Oriented Gradientes___ muy utilizado para la detección de objetos. Este descriptor computa un histograma de valores donde se indican la frecuencia con la que se dan las direcciones de los gradientes en pequeñas divisiones de la imagen. 

#### Descriptor Shape-Context


#### Descriptor de SIFT



### Enfoque de la implementación y eficiencia.

Para la implementación 